In [1]:
import optuna
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import StandardScaler

np.random.seed(42)
random_state = 42

scaler = StandardScaler()
# 加载数据集
data_ssgsea = pd.read_csv('ssgsea_10.csv')
X_ssgsea = data_ssgsea.iloc[:, 1:]
y_ssgsea = data_ssgsea.iloc[:, 0]  # 假设标签在 data1 的第一列

X_ssgsea_scaled = scaler.fit_transform(X_ssgsea)

# 拆分数据集
X_train_ssgsea, X_test_ssgsea, y_train_ssgsea, y_test_ssgsea = train_test_split(X_ssgsea_scaled, y_ssgsea, test_size=0.2, random_state = random_state)

data_ppi = pd.read_csv('ppi_10.csv')
X_ppi = data_ppi.iloc[:, 1:]
y_ppi = data_ppi.iloc[:, 0]  # 假设标签在 data1 的第一列

X_ppi_scaled = scaler.fit_transform(X_ppi)

# 拆分数据集
X_train_ppi, X_test_ppi, y_train_ppi, y_test_ppi = train_test_split(X_ppi_scaled, y_ppi, test_size=0.2, random_state = random_state)

data_wgcna = pd.read_csv('wgcna_10.csv')
X_wgcna = data_wgcna.iloc[:, 1:]
y_wgcna = data_wgcna.iloc[:, 0]  # 假设标签在 data1 的第一列

X_wgcna_scaled = scaler.fit_transform(X_wgcna)

# 拆分数据集
X_train_wgcna, X_test_wgcna, y_train_wgcna, y_test_wgcna = train_test_split(X_wgcna_scaled, y_wgcna, test_size=0.2, random_state = random_state)

In [6]:
import joblib
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from itertools import product

# 定义每个领域的模型
ssgsea_models = ['et', 'knn', 'logistic', 'xgb', 'rf',  'gnb', 'lda', 'ada', 'gb', 'svm']
ppi_models = ['et', 'knn', 'logistic', 'xgb', 'rf',  'gnb', 'lda', 'ada', 'gb', 'svm']
wgcna_models = ['et', 'knn', 'logistic', 'xgb', 'rf',  'gnb', 'lda', 'ada', 'gb', 'svm']

# 模型路径模板
model_path_template = r"C:\Users\一个大活人\5.8\ppi-wgcna-sssgsea-4.4\basemodel\{domain}\{model}_{domain}.joblib"

# 载入所有模型
models = {}

# 动态载入所有领域的模型
for domain in ['ssgsea', 'ppi', 'wgcna']:
    for model_name in locals()[f'{domain}_models']:  # 通过变量名获取每个领域的模型列表
        model_path = model_path_template.format(domain=domain, model=model_name)
        models[f'{model_name}_{domain}'] = joblib.load(model_path)

# 生成每个领域的模型组合
all_combinations = product(ssgsea_models, ppi_models, wgcna_models)

best_auc = 0
best_model = None
best_combination = None

# 遍历每个模型组合
for combination in all_combinations:
    # 获取每个模型的预测概率
    predictions_train = []
    predictions_test = []

    for domain, model_name in zip(['ssgsea', 'ppi', 'wgcna'], combination):
        model_key = f'{model_name}_{domain}'  # 动态组合领域和模型名
        model = models[model_key]

        # 训练模型并获取预测概率
        if domain == 'ssgsea':
            model.fit(X_train_ssgsea, y_train_ssgsea.squeeze())
            pred_train_ssgsea = model.predict_proba(X_train_ssgsea)[:, 1]
            pred_test_ssgsea = model.predict_proba(X_test_ssgsea)[:, 1]
        elif domain == 'ppi':
            model.fit(X_train_ppi, y_train_ppi.squeeze())
            pred_train_ppi = model.predict_proba(X_train_ppi)[:, 1]
            pred_test_ppi = model.predict_proba(X_test_ppi)[:, 1]
        elif domain == 'wgcna':
            model.fit(X_train_wgcna, y_train_wgcna.squeeze())
            pred_train_wgcna = model.predict_proba(X_train_wgcna)[:, 1]
            pred_test_wgcna = model.predict_proba(X_test_wgcna)[:, 1]

        # 将每个领域的预测结果添加到列表
        predictions_train.append(locals()[f'pred_train_{domain}'])
        predictions_test.append(locals()[f'pred_test_{domain}'])

    # 将所有模型的预测概率合并为新的特征
    X_train_combined = np.column_stack(predictions_train)
    X_test_combined = np.column_stack(predictions_test)

    # 训练逻辑回归模型
    lr_model = LogisticRegression()
    lr_model.fit(X_train_combined, y_train_ssgsea.squeeze())  # 使用ssgsea领域的y_train作为标签

    # 用训练好的模型进行预测
    y_pred_meta = lr_model.predict_proba(X_test_combined)[:, 1]

    # 计算AUC评分
    auc_score = roc_auc_score(y_test_ssgsea.squeeze(), y_pred_meta)  # 使用ssgsea领域的y_test作为标签

    print(f"组合 {combination} 的 AUC 分数: {auc_score}")

    # 更新最佳模型
    if auc_score > best_auc:
        best_auc = auc_score
        best_model = lr_model
        best_combination = combination

# 保存最佳模型
joblib.dump(best_model, 'best_model.joblib')

# 输出最佳组合和AUC评分
print(f"最佳模型的组合是: {best_combination}")
print(f"最佳 AUC 分数是: {best_auc}")


组合 ('et', 'et', 'et') 的 AUC 分数: 0.818840579710145
组合 ('et', 'et', 'knn') 的 AUC 分数: 0.8115942028985508
组合 ('et', 'et', 'logistic') 的 AUC 分数: 0.818840579710145
组合 ('et', 'et', 'xgb') 的 AUC 分数: 0.8297101449275363
组合 ('et', 'et', 'rf') 的 AUC 分数: 0.8514492753623188
组合 ('et', 'et', 'gnb') 的 AUC 分数: 0.822463768115942
组合 ('et', 'et', 'lda') 的 AUC 分数: 0.8224637681159421
组合 ('et', 'et', 'ada') 的 AUC 分数: 0.8260869565217391
组合 ('et', 'et', 'gb') 的 AUC 分数: 0.7898550724637681
组合 ('et', 'et', 'svm') 的 AUC 分数: 0.8152173913043478
组合 ('et', 'knn', 'et') 的 AUC 分数: 0.8115942028985508
组合 ('et', 'knn', 'knn') 的 AUC 分数: 0.8007246376811594
组合 ('et', 'knn', 'logistic') 的 AUC 分数: 0.8260869565217391
组合 ('et', 'knn', 'xgb') 的 AUC 分数: 0.8260869565217391
组合 ('et', 'knn', 'rf') 的 AUC 分数: 0.8369565217391305
组合 ('et', 'knn', 'gnb') 的 AUC 分数: 0.8188405797101449
组合 ('et', 'knn', 'lda') 的 AUC 分数: 0.8260869565217391
组合 ('et', 'knn', 'ada') 的 AUC 分数: 0.8152173913043478
组合 ('et', 'knn', 'gb') 的 AUC 分数: 0.8333333333333333
组合

组合 ('knn', 'gnb', 'rf') 的 AUC 分数: 0.753623188405797
组合 ('knn', 'gnb', 'gnb') 的 AUC 分数: 0.7101449275362319
组合 ('knn', 'gnb', 'lda') 的 AUC 分数: 0.717391304347826
组合 ('knn', 'gnb', 'ada') 的 AUC 分数: 0.7210144927536232
组合 ('knn', 'gnb', 'gb') 的 AUC 分数: 0.7318840579710145
组合 ('knn', 'gnb', 'svm') 的 AUC 分数: 0.7282608695652173
组合 ('knn', 'lda', 'et') 的 AUC 分数: 0.7355072463768115
组合 ('knn', 'lda', 'knn') 的 AUC 分数: 0.7246376811594204
组合 ('knn', 'lda', 'logistic') 的 AUC 分数: 0.7246376811594203
组合 ('knn', 'lda', 'xgb') 的 AUC 分数: 0.7463768115942029
组合 ('knn', 'lda', 'rf') 的 AUC 分数: 0.7753623188405797
组合 ('knn', 'lda', 'gnb') 的 AUC 分数: 0.7173913043478262
组合 ('knn', 'lda', 'lda') 的 AUC 分数: 0.7318840579710145
组合 ('knn', 'lda', 'ada') 的 AUC 分数: 0.7427536231884058
组合 ('knn', 'lda', 'gb') 的 AUC 分数: 0.7572463768115942
组合 ('knn', 'lda', 'svm') 的 AUC 分数: 0.7391304347826086
组合 ('knn', 'ada', 'et') 的 AUC 分数: 0.7173913043478262
组合 ('knn', 'ada', 'knn') 的 AUC 分数: 0.6956521739130435
组合 ('knn', 'ada', 'logistic') 的

组合 ('logistic', 'svm', 'gb') 的 AUC 分数: 0.7536231884057971
组合 ('logistic', 'svm', 'svm') 的 AUC 分数: 0.8876811594202899
组合 ('xgb', 'et', 'et') 的 AUC 分数: 0.7282608695652174
组合 ('xgb', 'et', 'knn') 的 AUC 分数: 0.7246376811594203
组合 ('xgb', 'et', 'logistic') 的 AUC 分数: 0.7463768115942029
组合 ('xgb', 'et', 'xgb') 的 AUC 分数: 0.7355072463768115
组合 ('xgb', 'et', 'rf') 的 AUC 分数: 0.7717391304347827
组合 ('xgb', 'et', 'gnb') 的 AUC 分数: 0.7282608695652174
组合 ('xgb', 'et', 'lda') 的 AUC 分数: 0.75
组合 ('xgb', 'et', 'ada') 的 AUC 分数: 0.7355072463768115
组合 ('xgb', 'et', 'gb') 的 AUC 分数: 0.7934782608695652
组合 ('xgb', 'et', 'svm') 的 AUC 分数: 0.75
组合 ('xgb', 'knn', 'et') 的 AUC 分数: 0.7753623188405797
组合 ('xgb', 'knn', 'knn') 的 AUC 分数: 0.7717391304347827
组合 ('xgb', 'knn', 'logistic') 的 AUC 分数: 0.7789855072463768
组合 ('xgb', 'knn', 'xgb') 的 AUC 分数: 0.7717391304347826
组合 ('xgb', 'knn', 'rf') 的 AUC 分数: 0.7789855072463768
组合 ('xgb', 'knn', 'gnb') 的 AUC 分数: 0.7753623188405797
组合 ('xgb', 'knn', 'lda') 的 AUC 分数: 0.778985507246376

组合 ('rf', 'gnb', 'knn') 的 AUC 分数: 0.605072463768116
组合 ('rf', 'gnb', 'logistic') 的 AUC 分数: 0.6811594202898551
组合 ('rf', 'gnb', 'xgb') 的 AUC 分数: 0.6557971014492754
组合 ('rf', 'gnb', 'rf') 的 AUC 分数: 0.7391304347826086
组合 ('rf', 'gnb', 'gnb') 的 AUC 分数: 0.5978260869565217
组合 ('rf', 'gnb', 'lda') 的 AUC 分数: 0.6811594202898551
组合 ('rf', 'gnb', 'ada') 的 AUC 分数: 0.6376811594202899
组合 ('rf', 'gnb', 'gb') 的 AUC 分数: 0.7427536231884058
组合 ('rf', 'gnb', 'svm') 的 AUC 分数: 0.6811594202898551
组合 ('rf', 'lda', 'et') 的 AUC 分数: 0.6992753623188406
组合 ('rf', 'lda', 'knn') 的 AUC 分数: 0.6956521739130435
组合 ('rf', 'lda', 'logistic') 的 AUC 分数: 0.7101449275362319
组合 ('rf', 'lda', 'xgb') 的 AUC 分数: 0.7463768115942029
组合 ('rf', 'lda', 'rf') 的 AUC 分数: 0.7789855072463768
组合 ('rf', 'lda', 'gnb') 的 AUC 分数: 0.6811594202898551
组合 ('rf', 'lda', 'lda') 的 AUC 分数: 0.7101449275362319
组合 ('rf', 'lda', 'ada') 的 AUC 分数: 0.717391304347826
组合 ('rf', 'lda', 'gb') 的 AUC 分数: 0.7536231884057971
组合 ('rf', 'lda', 'svm') 的 AUC 分数: 0.7137681

组合 ('lda', 'et', 'ada') 的 AUC 分数: 0.8514492753623188
组合 ('lda', 'et', 'gb') 的 AUC 分数: 0.8152173913043478
组合 ('lda', 'et', 'svm') 的 AUC 分数: 0.8478260869565217
组合 ('lda', 'knn', 'et') 的 AUC 分数: 0.8188405797101449
组合 ('lda', 'knn', 'knn') 的 AUC 分数: 0.8152173913043479
组合 ('lda', 'knn', 'logistic') 的 AUC 分数: 0.8369565217391304
组合 ('lda', 'knn', 'xgb') 的 AUC 分数: 0.8260869565217391
组合 ('lda', 'knn', 'rf') 的 AUC 分数: 0.8333333333333335
组合 ('lda', 'knn', 'gnb') 的 AUC 分数: 0.8188405797101449
组合 ('lda', 'knn', 'lda') 的 AUC 分数: 0.8369565217391304
组合 ('lda', 'knn', 'ada') 的 AUC 分数: 0.822463768115942
组合 ('lda', 'knn', 'gb') 的 AUC 分数: 0.8478260869565217
组合 ('lda', 'knn', 'svm') 的 AUC 分数: 0.8297101449275363
组合 ('lda', 'logistic', 'et') 的 AUC 分数: 0.8623188405797102
组合 ('lda', 'logistic', 'knn') 的 AUC 分数: 0.8442028985507247
组合 ('lda', 'logistic', 'logistic') 的 AUC 分数: 0.8369565217391305
组合 ('lda', 'logistic', 'xgb') 的 AUC 分数: 0.891304347826087
组合 ('lda', 'logistic', 'rf') 的 AUC 分数: 0.9094202898550725
组合 (

组合 ('ada', 'gnb', 'svm') 的 AUC 分数: 0.8478260869565217
组合 ('ada', 'lda', 'et') 的 AUC 分数: 0.8333333333333334
组合 ('ada', 'lda', 'knn') 的 AUC 分数: 0.782608695652174
组合 ('ada', 'lda', 'logistic') 的 AUC 分数: 0.8115942028985508
组合 ('ada', 'lda', 'xgb') 的 AUC 分数: 0.8623188405797102
组合 ('ada', 'lda', 'rf') 的 AUC 分数: 0.9057971014492754
组合 ('ada', 'lda', 'gnb') 的 AUC 分数: 0.8297101449275363
组合 ('ada', 'lda', 'lda') 的 AUC 分数: 0.8115942028985508
组合 ('ada', 'lda', 'ada') 的 AUC 分数: 0.8623188405797102
组合 ('ada', 'lda', 'gb') 的 AUC 分数: 0.7536231884057971
组合 ('ada', 'lda', 'svm') 的 AUC 分数: 0.8115942028985508
组合 ('ada', 'ada', 'et') 的 AUC 分数: 0.8478260869565217
组合 ('ada', 'ada', 'knn') 的 AUC 分数: 0.7971014492753623
组合 ('ada', 'ada', 'logistic') 的 AUC 分数: 0.8007246376811594
组合 ('ada', 'ada', 'xgb') 的 AUC 分数: 0.8405797101449275
组合 ('ada', 'ada', 'rf') 的 AUC 分数: 0.8804347826086958
组合 ('ada', 'ada', 'gnb') 的 AUC 分数: 0.7934782608695652
组合 ('ada', 'ada', 'lda') 的 AUC 分数: 0.7934782608695652
组合 ('ada', 'ada', 'ada')

组合 ('svm', 'knn', 'rf') 的 AUC 分数: 0.8369565217391305
组合 ('svm', 'knn', 'gnb') 的 AUC 分数: 0.8224637681159419
组合 ('svm', 'knn', 'lda') 的 AUC 分数: 0.8369565217391305
组合 ('svm', 'knn', 'ada') 的 AUC 分数: 0.8260869565217391
组合 ('svm', 'knn', 'gb') 的 AUC 分数: 0.8442028985507246
组合 ('svm', 'knn', 'svm') 的 AUC 分数: 0.8369565217391304
组合 ('svm', 'logistic', 'et') 的 AUC 分数: 0.855072463768116
组合 ('svm', 'logistic', 'knn') 的 AUC 分数: 0.8115942028985508
组合 ('svm', 'logistic', 'logistic') 的 AUC 分数: 0.8115942028985508
组合 ('svm', 'logistic', 'xgb') 的 AUC 分数: 0.8695652173913043
组合 ('svm', 'logistic', 'rf') 的 AUC 分数: 0.9239130434782609
组合 ('svm', 'logistic', 'gnb') 的 AUC 分数: 0.8333333333333334
组合 ('svm', 'logistic', 'lda') 的 AUC 分数: 0.818840579710145
组合 ('svm', 'logistic', 'ada') 的 AUC 分数: 0.8623188405797102
组合 ('svm', 'logistic', 'gb') 的 AUC 分数: 0.7572463768115942
组合 ('svm', 'logistic', 'svm') 的 AUC 分数: 0.8260869565217391
组合 ('svm', 'xgb', 'et') 的 AUC 分数: 0.782608695652174
组合 ('svm', 'xgb', 'knn') 的 AUC 分数: 0

In [3]:


# 输出逻辑回归模型的系数和截距
print("逻辑回归模型的系数：", best_model.coef_)
print("逻辑回归模型的截距：", best_model.intercept_)

# 如果你希望得到每个特征（即基础模型预测概率）对应的系数
feature_names = ['ssgsea', 'ppi', 'wgcna']
for feature, coef in zip(feature_names, best_model.coef_[0]):
    print(f"{feature} 的系数是: {coef}")



逻辑回归模型的系数： [[2.38453388 1.76074764 4.45668402]]
逻辑回归模型的截距： [-4.07206565]
ssgsea 的系数是: 2.384533877980014
ppi 的系数是: 1.7607476396710102
wgcna 的系数是: 4.456684021717978
